# HEK Quickstart

Run a minimal MLflow-backed evaluation and log HEK metadata.

In [ ]:
import mlflow
import pandas as pd
from sklearn.linear_model import LogisticRegression

from src.modules.evaluation import ModelEvaluator

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("hek-notebook-quickstart")

X = pd.DataFrame({"x1": [0, 0, 1, 1], "x2": [0, 1, 0, 1]})
y = pd.Series([0, 0, 1, 1])
model = LogisticRegression().fit(X, y)

In [ ]:
with mlflow.start_run(run_name="quickstart") as run:
    mlflow.set_tag("hokusai.eval_id", "quickstart-v1")
    mlflow.set_tag("hokusai.model_id", "demo-model")
    mlflow.set_tag("hokusai.primary_metric", "accuracy")
    mlflow.set_tag("hokusai.dataset.id", "toy-dataset")
    mlflow.set_tag("hokusai.dataset.hash", "sha256:" + "a" * 64)
    mlflow.set_tag("hokusai.dataset.num_samples", str(len(X)))

    evaluator = ModelEvaluator(metrics=["accuracy", "precision", "recall", "f1", "auroc"])
    metrics = evaluator.evaluate_sklearn_model(model, X, y)
    for metric_name, metric_value in metrics.items():
        if metric_value is not None:
            mlflow.log_metric(metric_name, float(metric_value))

print(run.info.run_id)
print(metrics)